In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
from tqdm.notebook import tqdm
from scipy.stats import ttest_ind


ayu_colors = {
    "axes.facecolor": "#1F2430",  # фон графика
    "figure.facecolor": "#1F2430",  # фон фигуры
    "text.color": "#CBCCC6",  # цвет текста
    "axes.labelcolor": "#CBCCC6",  # цвет подписей осей
    "xtick.color": "#707A8C",  # цвет делений оси X
    "ytick.color": "#707A8C",  # цвет делений оси Y
    "grid.color": "#232834",  # цвет сетки
    "axes.prop_cycle": plt.cycler(
        color=["#8dd3c7", "#feffb3", "#bfbbd9", "#fa8174", "#81b1d2"]
    ),  # цвета линий как в dark_background
}

plt.rcParams.update(ayu_colors)

In [15]:
cards = np.random.binomial(1, 0.5, size=10)
guess = np.random.binomial(1, 0.5, size=10)

In [21]:
int((guess == cards).sum())

2

In [31]:
result = []

for _ in range(1000):
    cards = np.random.binomial(1, 0.5, size=10)
    guess = np.random.binomial(1, 0.5, size=10)
    if all(cards == guess):
        result.append(10)
    elif int((guess == cards).sum()) == 9:
        result.append(9)
    else:
        continue

In [36]:
len(result) / 1000

0.009

test_1 \
control_1 \
test_2 \
control_2 

При увеличении количества экспериментов повышается вероятность того, что хотя бы один эксперимент будет успешным по чистой случайности. \
Проверим эту гипотезу можно по формуле $1-(1-\alpha)^{n}$, где $n$ - количество экспериментов и с помощью monte-carlo

In [59]:
from statsmodels.stats.proportion import proportions_ztest


def experiment(baseline, size, effect=None):
    if effect is None:
        a = np.random.binomial(1, baseline, size)
        b = np.random.binomial(1, baseline, size)
    else:
        a = np.random.binomial(1, baseline, size)
        b = np.random.binomial(1, baseline + effect, size)
    _, pval = proportions_ztest(
        [a.sum(), b.sum()], [len(a), len(b)], alternative="two-sided"
    )

    return pval

In [64]:
np.mean(
    [
        any([experiment(0.2, 1030) < 0.05 for _ in range(2)])
        for _ in tqdm(range(10000))
    ]
)

  0%|          | 0/10000 [00:00<?, ?it/s]

np.float64(0.0985)

In [66]:
1 - (1 - 0.05) ** 2

0.09750000000000003

Воспользуемся поправкой Бонферрони:

In [67]:
np.mean(
    [
        any([experiment(0.2, 1030) < (0.05 / 2) for _ in range(2)])
        for _ in tqdm(range(10000))
    ]
)

  0%|          | 0/10000 [00:00<?, ?it/s]

np.float64(0.0506)

Но снижая $\alpha$ мы снижаем и мощность наших тестов